# Ferrofluid-Monolayer - Part III 

## Susceptibility with fluctuation formulas

In this part we want to calculate estimators for the initial susceptibility, i.e. the susceptibility at zero external magnetic field. One could carry out several simulations with different external magnetic field strenghts and get the initial susceptibility by fitting a line to the results. We want to go a more elegant way by using fluctuation formulas known from statistical mechanics.
In three dimensions the initial susceptibility $\chi_{init}$ can be calculated with zero field simulations through

\begin{equation}
    \chi_{init} = \frac{V \cdot \mu_0}{3 \cdot k_B T} \left( \langle \boldsymbol{M}^2 \rangle - \langle \boldsymbol{M} \rangle^2 \right) = \frac{\mu_0}{3 \cdot k_B T \cdot V} \left( \langle \boldsymbol{\mu}_{tot}^2 \rangle - \langle \boldsymbol{\mu}_{tot} \rangle^2 \right)
\end{equation}

where $\boldsymbol{M}$ is the magnetization vector. In direction $i$ it reads

\begin{equation}
    M_i = \frac{1}{V} \Bigg\langle \sum_{j=1}^N \mu_j^i \Bigg\rangle
\end{equation}

where $\mu_j^i$ is the $j$ th dipole moment in direction $i$.

For a quasi two dimensional System due to symmetry we have at least two different susceptibilities

\begin{equation}
    \chi_{init,||} = \frac{V \cdot \mu_0}{2 \cdot k_B T} \left( \langle \boldsymbol{M}^2_{||} \rangle - \langle \boldsymbol{M}_{||} \rangle^2 \right) = \frac{\mu_0}{2 \cdot k_B T \cdot V} \left( \langle \boldsymbol{\mu}^2_{tot,||} \rangle - \langle \boldsymbol{\mu}_{tot,||} \rangle^2 \right)
\end{equation}

and 

\begin{equation}
    \chi_{init,\perp} = \frac{V \cdot \mu_0}{k_B T} \left( \langle \boldsymbol{M}^2_{\perp} \rangle - \langle \boldsymbol{M}_{\perp} \rangle^2 \right) = \frac{\mu_0}{k_B T \cdot V} \left( \langle \boldsymbol{\mu}^2_{tot,\perp} \rangle - \langle \boldsymbol{\mu}_{tot,\perp} \rangle^2 \right)
\end{equation}

where $\boldsymbol{\mu}_{tot,||}$ and $\boldsymbol{\mu}_{tot,\perp}$ are the dipole moments of the whole system parallel and perpendicular to the monolayer plane.  
We now want to calculate and compare the initial susceptibility $\chi_{init}$ with the results of Ref. [1] for the monodisperse case. As the system itself is homogen we have the two different susceptibilities $\chi_{init,||}$ and $\chi_{init,\perp}$.

First we import all necessary packages

In [1]:
from __future__ import print_function

import espressomd

from espressomd.magnetostatics import DipolarP3M
from espressomd.magnetostatic_extensions import DLC

from espressomd.cluster_analysis import ClusterStructure
from espressomd.pair_criteria import DistanceCriterion


import numpy as np

Now we setting up all necessary simulation parameters

In [2]:
lj_sigma=1
lj_epsilon=1
lj_cut = 2**(1./6.) * lj_sigma

# magnetic field constant
mu_0 = 4. * np.pi

# Particles
N = 1000

# density
rho = 0.4
# Area fraction of the mono-layer
phi = rho * np.pi * lj_sigma**2. / 4.

# Dipolar interaction parameter lambda = mu_0 m^2 /(4 pi sigma^3 kT)
dip_lambda = 1.

# Temperature
kT =1.0

# Friction coefficient
gamma=1.0

# Time step
dt =0.01

# Verlet list skin
skin =0.4

# System setup
box_size = (N * np.pi * (lj_sigma/2.)**2 /phi)**0.5

# dipole moment
dipm = np.sqrt(dip_lambda*4*np.pi*lj_sigma**3*kT / mu_0)

and the system (for more details see Part I)

In [3]:
system=espressomd.System(box_l=(box_size,box_size,box_size)) 
system.time_step=dt
system.cell_system.skin=skin
system.thermostat.set_langevin(kT=kT,gamma=gamma)
system.set_random_state_PRNG()

# Lennard Jones interaction
system.non_bonded_inter[0,0].lennard_jones.set_params(epsilon=lj_epsilon,sigma=lj_sigma,cutoff=lj_cut, shift="auto")

# Random dipole moments
dip_phi=np.random.random((N,1)) *2. * np.pi
dip_cos_theta = 2*np.random.random((N,1)) -1
dip_sin_theta = np.sin(np.arccos(dip_cos_theta))
dip = np.hstack((
   dip_sin_theta *np.sin(dip_phi),
   dip_sin_theta *np.cos(dip_phi),
   dip_cos_theta))

# Random positions in the monolayer
pos = box_size* np.hstack((np.random.random((N,2)), np.zeros((N,1))))

# Add particles
system.part.add(pos=pos,rotation=N*[(1,1,1)],dip=dip,fix=N*[(0,0,1)])

# Remove overlap between particles by means of the steepest descent method
system.integrator.set_steepest_descent(
    f_max=0,gamma=0.1,max_displacement=0.05)

while system.analysis.energy()["total"] > 5*kT*N:
    system.integrator.run(20)

# Switch to velocity Verlet integrator
system.integrator.set_vv()

# Setup dipolar P3M and dipolar layer correction
system.actors.add(DipolarP3M(accuracy=5E-4,prefactor=dip_lambda*lj_sigma**3*kT))
system.actors.add(DLC(maxPWerror=1E-4, gap_size=box_size-lj_sigma))

Dipolar P3M tune parameters: Accuracy goal = 5.00000e-04 prefactor = 1.00000e+00
System: box_l = 5.00000e+01 # charged part = 1000 Sum[q_i^2] = 1.00000e+03
Dmesh cao Dr_cut_iL   Dalpha_L     Derr         Drs_err    Dks_err    time [ms]
8    3   2.29664e-01 7.91354e+00 4.94976e-04 3.535e-04 3.464e-04 10      
8    2   2.43117e-01 7.17458e+00 4.97684e-04 3.535e-04 3.503e-04 11      
8    4   2.22938e-01 8.31450e+00 4.98869e-04 3.535e-04 3.520e-04 11      
8    1   2.81555e-01 5.41789e+00 4.95805e-04 3.536e-04 3.476e-04 14      
16   3   1.80322e-01 1.15365e+01 4.97737e-04 3.536e-04 3.503e-04 7       
16   2   1.99162e-01 9.94221e+00 4.98121e-04 3.535e-04 3.509e-04 8       
16   4   1.72248e-01 1.23288e+01 4.96857e-04 3.536e-04 3.491e-04 8       
16   1   2.29664e-01 7.91354e+00 7.75162e-04 3.535e-04 6.898e-04 accuracy not achieved
32   3   1.33128e-01 1.75927e+01 4.91338e-04 3.535e-04 3.412e-04 15      
32   2   1.58486e-01 1.38719e+01 4.94693e-04 3.535e-04 3.460e-04 16      
32   4   1.

Now we equilibrate for a while

In [4]:
print("Equilibration...")
for i in range(30):
  system.integrator.run(1000)
  print("Dipolar energy: %.2f" % system.analysis.energy()["dipolar"])
  print("progress: {:6.2f} %".format((float(i)+1)/30 * 100))
print("Equilibration done")

Equilibration...
Dipolar energy: -302.30
progress:   3.33 %
Dipolar energy: -303.22
progress:   6.67 %
Dipolar energy: -280.77
progress:  10.00 %
Dipolar energy: -287.00
progress:  13.33 %
Dipolar energy: -246.92
progress:  16.67 %
Dipolar energy: -280.59
progress:  20.00 %
Dipolar energy: -272.56
progress:  23.33 %
Dipolar energy: -291.62
progress:  26.67 %
Dipolar energy: -278.53
progress:  30.00 %
Dipolar energy: -281.51
progress:  33.33 %
Dipolar energy: -298.94
progress:  36.67 %
Dipolar energy: -298.54
progress:  40.00 %
Dipolar energy: -314.08
progress:  43.33 %
Dipolar energy: -277.32
progress:  46.67 %
Dipolar energy: -299.37
progress:  50.00 %
Dipolar energy: -292.87
progress:  53.33 %
Dipolar energy: -282.04
progress:  56.67 %
Dipolar energy: -268.73
progress:  60.00 %
Dipolar energy: -270.00
progress:  63.33 %
Dipolar energy: -284.49
progress:  66.67 %
Dipolar energy: -253.31
progress:  70.00 %
Dipolar energy: -283.94
progress:  73.33 %
Dipolar energy: -281.39
progress:  76

As we need the magnetization of our system, we import <tt>MagneticDipoleMoment</tt> from <tt>observables</tt> which returns us the total dipole moment of the system which is the magnetization times the volume of the system.

In [5]:
from espressomd.observables import MagneticDipoleMoment
dipm_tot_calc = MagneticDipoleMoment(ids=system.part[:].id)

Now we set the desired number of loops for the sampling

In [6]:
# Sampling
loops = 1000

and sample the first and second moment of the magnetization or total dipole moment, by averaging over all total dipole moments occuring during the simulation

In [8]:
print('Sampling ...')
# calculate initial total dipole moment
dipm_tot_temp = dipm_tot_calc.calculate()

# initiate variables for sum of first and second moment of total dipole moment
dipm_tot_sum = dipm_tot_temp
dipm_tot_2_sum = np.square(dipm_tot_temp)

# sample dipole moment
for i in range(loops):
    system.integrator.run(100)
    dipm_tot_temp = dipm_tot_calc.calculate()
    
    # add current dipole moment
    dipm_tot_sum = np.sum((dipm_tot_sum, dipm_tot_temp), axis=0)
    dipm_tot_2_sum = np.sum((dipm_tot_2_sum, np.square(dipm_tot_temp)), axis=0)
    print("progress: {:6.2f} %".format((float(i)+1)/loops * 100))
    
# calculate average first and second moment of total dipole moment
dipm_tot = dipm_tot_sum / loops
dipm_tot_2 = dipm_tot_2_sum / loops
print("Sampling done")

Sampling ...
progress:   3.33 %
progress:   6.67 %
progress:  10.00 %
progress:  13.33 %
progress:  16.67 %
progress:  20.00 %
progress:  23.33 %
progress:  26.67 %
progress:  30.00 %
progress:  33.33 %
progress:  36.67 %
progress:  40.00 %
progress:  43.33 %
progress:  46.67 %
progress:  50.00 %
progress:  53.33 %
progress:  56.67 %
progress:  60.00 %
progress:  63.33 %
progress:  66.67 %
progress:  70.00 %
progress:  73.33 %
progress:  76.67 %
progress:  80.00 %
progress:  83.33 %
progress:  86.67 %
progress:  90.00 %
progress:  93.33 %
progress:  96.67 %
progress: 100.00 %
progress: 103.33 %
progress: 106.67 %
progress: 110.00 %
progress: 113.33 %
progress: 116.67 %
progress: 120.00 %
progress: 123.33 %
progress: 126.67 %
progress: 130.00 %
progress: 133.33 %
progress: 136.67 %
progress: 140.00 %
progress: 143.33 %
progress: 146.67 %
progress: 150.00 %
progress: 153.33 %
progress: 156.67 %
progress: 160.00 %
progress: 163.33 %
progress: 166.67 %
progress: 170.00 %
progress: 173.33 %

progress: 1416.67 %
progress: 1420.00 %
progress: 1423.33 %
progress: 1426.67 %
progress: 1430.00 %
progress: 1433.33 %
progress: 1436.67 %
progress: 1440.00 %
progress: 1443.33 %
progress: 1446.67 %
progress: 1450.00 %
progress: 1453.33 %
progress: 1456.67 %
progress: 1460.00 %
progress: 1463.33 %
progress: 1466.67 %
progress: 1470.00 %
progress: 1473.33 %
progress: 1476.67 %
progress: 1480.00 %
progress: 1483.33 %
progress: 1486.67 %
progress: 1490.00 %
progress: 1493.33 %
progress: 1496.67 %
progress: 1500.00 %
progress: 1503.33 %
progress: 1506.67 %
progress: 1510.00 %
progress: 1513.33 %
progress: 1516.67 %
progress: 1520.00 %
progress: 1523.33 %
progress: 1526.67 %
progress: 1530.00 %
progress: 1533.33 %
progress: 1536.67 %
progress: 1540.00 %
progress: 1543.33 %
progress: 1546.67 %
progress: 1550.00 %
progress: 1553.33 %
progress: 1556.67 %
progress: 1560.00 %
progress: 1563.33 %
progress: 1566.67 %
progress: 1570.00 %
progress: 1573.33 %
progress: 1576.67 %
progress: 1580.00 %


progress: 2783.33 %
progress: 2786.67 %
progress: 2790.00 %
progress: 2793.33 %
progress: 2796.67 %
progress: 2800.00 %
progress: 2803.33 %
progress: 2806.67 %
progress: 2810.00 %
progress: 2813.33 %
progress: 2816.67 %
progress: 2820.00 %
progress: 2823.33 %
progress: 2826.67 %
progress: 2830.00 %
progress: 2833.33 %
progress: 2836.67 %
progress: 2840.00 %
progress: 2843.33 %
progress: 2846.67 %
progress: 2850.00 %
progress: 2853.33 %
progress: 2856.67 %
progress: 2860.00 %
progress: 2863.33 %
progress: 2866.67 %
progress: 2870.00 %
progress: 2873.33 %
progress: 2876.67 %
progress: 2880.00 %
progress: 2883.33 %
progress: 2886.67 %
progress: 2890.00 %
progress: 2893.33 %
progress: 2896.67 %
progress: 2900.00 %
progress: 2903.33 %
progress: 2906.67 %
progress: 2910.00 %
progress: 2913.33 %
progress: 2916.67 %
progress: 2920.00 %
progress: 2923.33 %
progress: 2926.67 %
progress: 2930.00 %
progress: 2933.33 %
progress: 2936.67 %
progress: 2940.00 %
progress: 2943.33 %
progress: 2946.67 %


Now we can calculate estimators of the susceptibility parallel and perpendicular to the monolayer plane

In [9]:
# susceptibility parallel to monolayer plane
chi_p =  mu_0 /(box_size**2. * 2.) * ( (dipm_tot_2[0] * dipm**2. - (dipm_tot[0] * dipm)**2.) + (dipm_tot_2[1] * dipm**2. - (dipm_tot[1] * dipm)**2.) )

In [10]:
# susceptibility perpendicular to monolayer plane
chi_perp = mu_0 /(box_size**2.) * (dipm_tot_2[2] * dipm**2. - (dipm_tot[2] * dipm)**2.)

and print the results

In [11]:
print('chi_parallel\t\t= {}'.format(chi_p))
print('chi_perpendicular\t= {}'.format(chi_perp))

chi_parallel		= 2.84447188102
chi_perpendicular	= 0.822560450483


Copared with table 1 in Ref. [1] they should be very similar

[1] Tamás Kristóf and István Szalai. “Magnetic properties in monolayers of a model
polydisperse ferrofluid”. In: Phys. Rev. E 72 (4 Oct. 2005), p. 041105. doi: 10.1103/
PhysRevE.72.041105. url: https://link.aps.org/doi/10.1103/PhysRevE.72.
041105.